In [2]:
!pip install einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 419.3 kB/s eta 0:00:000:00:01


In [18]:
# Load model directly
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

/opt/homebrew/Caskroom/miniforge/base/envs/asrDevMyNorth/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<All keys matched successfully>


In [32]:
sql1= r"""SELECT employees.name, departments.department_name
                FROM employees"""
sql2= r"""SELECT name
                  FROM departments """

In [33]:
embed1= model.encode(sql1)
embed2= model.encode(sql2)

In [34]:
# Calculate cosine similarity between embed1 and embed2
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity([embed1], [embed2])
print(similarity)


[[0.84708035]]


In [35]:
from pglast import parse_sql
import pprint

ast1= parse_sql(sql1)[0].stmt
ast2 = parse_sql(sql2)[0].stmt
pp = pprint.PrettyPrinter()
ast = {
    'AST1': pp.pformat(ast1(skip_none=True)),
    'AST2': pp.pformat(ast2(skip_none=True))
}

In [36]:
ast

{'AST1': "{'@': 'SelectStmt',\n 'all': False,\n 'fromClause': ({'@': 'RangeVar',\n                 'inh': True,\n                 'location': 72,\n                 'relname': 'employees',\n                 'relpersistence': 'p'},),\n 'groupDistinct': False,\n 'limitOption': {'#': 'LimitOption',\n                 'name': 'LIMIT_OPTION_DEFAULT',\n                 'value': 0},\n 'op': {'#': 'SetOperation', 'name': 'SETOP_NONE', 'value': 0},\n 'targetList': ({'@': 'ResTarget',\n                 'location': 7,\n                 'val': {'@': 'ColumnRef',\n                         'fields': ({'@': 'String', 'sval': 'employees'},\n                                    {'@': 'String', 'sval': 'name'}),\n                         'location': 7}},\n                {'@': 'ResTarget',\n                 'location': 23,\n                 'val': {'@': 'ColumnRef',\n                         'fields': ({'@': 'String', 'sval': 'departments'},\n                                    {'@': 'String', 'sval': 'dep

In [38]:
embed1ast= model.encode(ast["AST1"])
embed2ast= model.encode(ast["AST2"])
similarity = cosine_similarity([embed1ast], [embed2ast])
print(f'AST similarity  : {similarity}')

AST similarity  : [[0.98262715]]
